In [1]:
import re
import polars as pl
pl.Config.set_fmt_str_lengths(50)
print(pl.__version__)

from utils import xmp_to_taglist

0.20.6


In [2]:
col_taxon_file = "/home/wsyxbcl/Downloads/col-20231215/NameUsage.tsv"
col_name_file = "/home/wsyxbcl/Downloads/col-20231215/VernacularName.tsv"
shanshui_file = "/home/wsyxbcl/Downloads/_Species_Data__202401051603.csv"
gbif_backbone_file = "/home/wsyxbcl/Downloads/gbif-backbone/Taxon.tsv"
gbif_backbone_name_file = "/home/wsyxbcl/Downloads/gbif-backbone/VernacularName.tsv"


In [3]:
dtypes = {
    # "col:ID": pl.Categorical,
    "col:rank": pl.Categorical,
    "col:status": pl.Categorical,
}
# COL taxon data
df_col_taxon_raw = pl.read_csv(col_taxon_file, separator='\t', infer_schema_length=20000, rechunk=True, quote_char=None)
df_col_taxon = df_col_taxon_raw.filter(pl.col("col:code").eq("zoological"))
df_col_taxon = df_col_taxon.select([
    pl.col("col:ID").alias("taxonID"), 
    pl.col("col:parentID").alias("parentID"), 
    pl.col("col:rank"), 
    pl.col("col:scientificName").str.replace(r' \([^)]*\)', '').alias("scientificName"), 
    pl.col("col:status").alias("status")])
# COL name data
df_col_name_raw = pl.read_csv(col_name_file, separator="\t", infer_schema_length=20000)
df_col_name = df_col_name_raw.filter(pl.col("col:language") == "eng")
df_col_name = df_col_name.select([
    pl.col("col:taxonID").alias("taxonID"),
    pl.col("col:name").str.to_lowercase().alias("colVernacularNameEN"),
])
df_col_name = df_col_name.group_by("taxonID").agg([pl.col("colVernacularNameEN")])

df_col = df_col_taxon.join(df_col_name, on="taxonID", how="left")

In [4]:
# GBIF backbone data

df_gbif_taxon_raw = pl.read_csv(gbif_backbone_file, separator='\t', rechunk=True, quote_char=None)
# df_gbif_taxon = df_gbif_taxon_raw.filter(pl.col("taxonRank").is_in(["species", "subspecies"]))
df_gbif_taxon = (df_gbif_taxon_raw
    .filter(pl.col("kingdom").eq("Animalia"))
    .filter(pl.col("taxonRank").is_in(["species", "subspecies"]))
    .filter(pl.col("taxonomicStatus").eq("accepted"))
    .select([
        pl.col("taxonID").alias("gbifTaxonID"),
        pl.col("canonicalName").alias("scientificName"),
    ])
)

df_gbif_name_raw = pl.read_csv(gbif_backbone_name_file, separator="\t")
df_gbif_name = (df_gbif_name_raw
    .filter((pl.col("language") == "en") | (pl.col("language").is_null()))
    .filter(pl.col("source") != "United Kingdom Species Inventory (UKSI)")
    .select([
        pl.col("taxonID").alias("gbifTaxonID"),
        pl.col("vernacularName").str.to_lowercase().alias("gbifVernacularNameEN"),])
    .group_by("gbifTaxonID")
    .agg([pl.col("gbifVernacularNameEN")])
)

df_gbif = df_gbif_taxon.join(df_gbif_name, on="gbifTaxonID", how="left")

In [60]:
# df_gbif_name_raw.get_column("source").unique().to_list()

['Truchas Mexicanas',
 'Catalogue of the Lepidoptera of Belgium',
 'Checklist of the Butterflies (Papilionoidea) of Vermont, USA',
 'Taxon list of Orthoptera (Grashoppers) from Germany compiled at the SNSB',
 'Herpestidae',
 'Iniidae',
 'Calomyscidae',
 'Octodontidae',
 'Pelagic Invertebrates',
 'Acrobatidae',
 'Orthoptera Species File',
 'Castoridae',
 'Petromuridae',
 'Ziphiidae',
 'Mantophasmatodea Species File',
 'Platacanthomyidae',
 'Myrmecophagidae',
 'BioAcoustica: Wildlife Sounds Database',
 'Myzopodidae',
 'Cataloging Diversity in the Sacoglossa',
 'Terrestrial Slugs Web',
 'Checklist of the marine and estuarine fishes of New Ireland Province, Papua New Guinea, western Pacific Ocean, with 810 new records',
 'Taxon list of Pscoptera from Germany compiled in the context of the GBOL project',
 'Rhinonycteridae',
 'Mammals of the World',
 'Abrocomidae',
 'Lista de plantas útiles y promisorias en la Comunidad de Wacurabá (Caño Cuduyarí) en el Departamento de Vaupés (Amazonía Colom

In [5]:
# Taglist
taglist_yushu = xmp_to_taglist("/home/wsyxbcl/Downloads/玉树TagList.jpg.xmp")
taglist_qilian = xmp_to_taglist("/home/wsyxbcl/Downloads/祁连TagList.jpg.xmp")
taglist = list((set(taglist_qilian) | set(taglist_yushu)))
s_tag = pl.Series("tag", taglist).str.to_lowercase()

In [63]:
# # find unincluded tag in COL vernacular name
# tag_unincluded_in_col = []
# for tag in taglist:
#     try:
#         df_col.filter(pl.col("colVernacularNameEN").list.contains(tag.lower())).get_column("scientificName")[0]
#     except IndexError:
#         tag_unincluded_in_col.append(tag)
# print(tag_unincluded_in_col)

# # find unincluded tag in GBIF vernacular name
# tag_unincluded_in_gbif = []
# for tag in taglist:
#     try:
#         df_gbif.filter(pl.col("gbifVernacularNameEN").list.contains(tag.lower())).get_column("scientificName")[0]
#     except IndexError:
#         tag_unincluded_in_gbif.append(tag)
# print(tag_unincluded_in_gbif)

['White-winged redstart', 'Rufous-necked snowfinch', 'Fox spp.', 'Black-winged snowfinch', 'Altai weasel', "White's thrush", 'Bat spp.', 'Eurasian eagle owl', 'White-lipped deer', 'Unidentified', 'Domestic camel', 'Ground tit', 'Small rodents', 'Tibetan brown bear', 'Himalayan griffon', 'Japanese tit', "Blanford's snowfinch", 'Blue sheep', 'Black-rumped magpie', 'Domestic horse', 'Sambar deer', 'Useless data', 'Pika spp.', 'White-rumped snowfinch', 'Stone marten', 'Common leopard cat', 'White-capped water redstart', 'Bird spp.', 'Blank', 'Chough', 'Common leopard', 'Red-eared pika']


In [6]:
# Check tag in GBIF backbone and output for edit
tag_list = []
latin_name_list = []
for tag in taglist:
    tag_list.append(tag)
    try:
        latin_name_list.append(df_gbif.filter(pl.col("gbifVernacularNameEN").list.contains(tag.lower())).get_column("scientificName").to_list())
    except IndexError:
        latin_name_list.append([])
    

In [7]:
# COL patch
latin_name_patched = []
for (i, names) in enumerate(latin_name_list):
    if len(names) > 1:
        # print(names)
        names_patched = []
        for name in names:
            if df_col.filter(pl.col("scientificName").eq(name)).is_empty():
                pass
            else:
                names_patched.append(name)
        latin_name_patched.append(names_patched)
    else:
        latin_name_patched.append(latin_name_list[i])

In [8]:
# output for edit
import csv
with open('taglist_to_be_edit.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(tag_list, ['|'.join(names) for names in latin_name_patched]))

In [97]:
# Manual patch

In [98]:
df_col.filter(pl.col("scientificName").eq("Chaimarrornis leucocephalus"))

taxonID,parentID,col:rank,scientificName,status,colVernacularNameEN
str,str,str,str,str,list[str]
"""TF6L""","""62M2T""","""species""","""Chaimarrornis leucocephalus""","""accepted""","[""white-capped redstart""]"


In [99]:
df_col.filter(pl.col("scientificName").eq("Panthera pardus"))

taxonID,parentID,col:rank,scientificName,status,colVernacularNameEN
str,str,str,str,str,list[str]
"""4CGXR""","""6DBT""","""species""","""Panthera pardus""","""accepted""","[""leopard""]"


In [108]:
df_col.filter(pl.col("scientificName").eq("Camelus ferus"))

taxonID,parentID,col:rank,scientificName,status,colVernacularNameEN
str,str,str,str,str,list[str]
"""Q9XF""","""Q9XD""","""species""","""Camelus ferus""","""ambiguous synonym""",null
"""5WWKW""","""7JFKC""","""species""","""Camelus ferus""","""ambiguous synonym""",null


In [111]:
df_col.filter(pl.col("taxonID").eq("4N9HG"))

taxonID,parentID,col:rank,scientificName,status,colVernacularNameEN
str,str,str,str,str,list[str]
"""4N9HG""","""6Y7B""","""species""","""Przewalskium albirostre""","""accepted""",null


In [113]:
df_col.filter(pl.col("scientificName").eq("Cervus albirostris"))

taxonID,parentID,col:rank,scientificName,status,colVernacularNameEN
str,str,str,str,str,list[str]
"""T569""","""4N9HG""","""species""","""Cervus albirostris""","""synonym""",null


In [114]:
df_col.filter(pl.col("scientificName").eq("Ursus arctos pruinosus"))

taxonID,parentID,col:rank,scientificName,status,colVernacularNameEN
str,str,str,str,str,list[str]
"""BP6VT""","""7F2KB""","""subspecies""","""Ursus arctos pruinosus""","""accepted""","[""brown bear""]"


In [9]:
df_taglist_patched = pl.read_csv("./taglist_latin_name_patched.csv", has_header=False).select([pl.col("column_1").alias("tag"), pl.col("column_2").alias("scientificName")])

In [74]:
df_taglist_patched

tag,scientificName
str,str
"""Domestic cat""","""Felis catus"""
"""Olive-backed pipit""","""Anthus hodgsoni"""
"""Blue-fronted redstart""","""Phoenicurus frontalis"""
"""Chukar partridge""","""Alectoris chukar"""
"""Red-eared pika""","""Ochotona erythrotis"""
"""Eurasian eagle owl""","""Bubo bubo"""
"""Glover's pika""","""Ochotona gloveri"""
"""Domestic yak""","""Bos grunniens"""
"""Eurasian hoopoe""","""Upupa epops"""


In [10]:
df_shanshui_raw = pl.read_csv(shanshui_file)
# Patch
iucn_patch_dict = {
    'LR/nt': 'NT',
    'LR/LC': 'LC'
}
rcb_patch_dict = {
    'Doubt:NT,VU,EN,CR': None
}
nationl_patch_dict = {
    '?': None,
    'Ⅰ': 'I'
}

df_shanshui = df_shanshui_raw.select([
    pl.col("SpeciesID").alias("shanshuiTaxonID"),
    pl.col("ChineseName").alias("shanshuiVernacularNameCN"),
    pl.col("ScientificName").alias("scientificName"),
    pl.col("IUCN").replace(iucn_patch_dict).alias("protectLevelIUCN"),
    pl.col("RCB").replace(rcb_patch_dict).alias("protectLevelRCB"),
    pl.col("NationalKey").replace(nationl_patch_dict).alias("protectLevelNationalKey")
])

In [11]:
# Patch scientific name in shanshui data to COL
df_shanshui_patch = df_shanshui.join(df_col, on="scientificName", how="left")

In [15]:
df_shanshui_synonym = df_shanshui_patch.filter(pl.col("status").eq("synonym"))
synonym_parent_id = df_shanshui_synonym.get_column("parentID")
synonym_scientific_names = []
for id in synonym_parent_id:
    synonym_scientific_names.append(df_col.filter(pl.col("taxonID").eq(id)).get_column("scientificName")[0])

df_shanshui_synonym = df_shanshui_synonym.with_columns([
    pl.Series(synonym_scientific_names).alias("scientificName"),
    pl.lit("accepted").alias("status")])


In [25]:
df_shanshui = pl.concat([
    df_shanshui_patch.filter(pl.col("status").ne("synonym")),
    df_shanshui_synonym]).select([
        pl.col("shanshuiTaxonID"),
        pl.col("shanshuiVernacularNameCN"),
        pl.col("scientificName"),
        pl.col("protectLevelIUCN"),
        pl.col("protectLevelRCB"),
        pl.col("protectLevelNationalKey")
    ])
df_shanshui

shanshuiTaxonID,shanshuiVernacularNameCN,scientificName,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey
str,str,str,str,str,str
"""AM0009""","""台湾小鲵""","""Hynobius formosanus""","""EN""","""EN""","""II"""
"""AM0012""","""楚南小鲵""","""Hynobius sonani""","""EN""","""EN""","""II"""
"""AM0021""","""辽宁爪鲵""","""Onychodactylus zhaoermii""",null,"""CR""","""I"""
"""AM0013""","""商城肥鲵""","""Pachyhynobius shangchengensis""","""VU""","""VU""",null
"""AM0022""","""吉林爪鲵""","""Onychodactylus zhangyapingi""","""DD""","""VU""","""II"""
"""AM0023""","""巫山巴鲵""","""Liua shihi""","""LC""","""NT""","""II"""
"""AM0025""","""新疆北鲵""","""Ranodon sibiricus""","""EN""","""CR""","""II"""
"""AM0028""","""龙洞山溪鲵""","""Batrachuperus londongensis""","""EN""","""VU""","""II"""
"""AM0002""","""安吉小鲵""","""Hynobius amjiensis""","""CR""","""CR""","""I"""


In [26]:
df_taglist_patched.join(df_shanshui, on="scientificName", how="left").filter(pl.col("shanshuiVernacularNameCN").is_null())

tag,scientificName,shanshuiTaxonID,shanshuiVernacularNameCN,protectLevelIUCN,protectLevelRCB,protectLevelNationalKey
str,str,str,str,str,str,str
"""Domestic cat""","""Felis catus""",null,null,null,null,null
"""Blue-fronted redstart""","""Phoenicurus frontalis""",null,null,null,null,null
"""Domestic yak""","""Bos grunniens""",null,null,null,null,null
"""Himalayan buzzard""","""Buteo burmanicus""",null,null,null,null,null
"""Japanese tit""","""Parus minor""",null,null,null,null,null
"""White-throated redstart""","""Phoenicurus schisticeps""",null,null,null,null,null
"""Kiang""","""Equus kiang""",null,null,null,null,null
"""Black-rumped magpie""","""Pica bottanensis""",null,null,null,null,null
"""Tibetan brown bear""","""Ursus arctos pruinosus""",null,null,null,null,null
